## Imports

In [224]:
import pandas as pd
import numpy as np
import json
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta

In [225]:
#Load excel data
exp = pd.json_normalize(pd.read_json('../data/exp.json')['expenses'])

In [226]:
exp

,title,cadence,date_added,first_due,amount
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0


In [227]:
#Check dtypes and update if needed
exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1 non-null      object 
 1   cadence     1 non-null      float64
 2   date_added  1 non-null      object 
 3   first_due   1 non-null      object 
 4   amount      1 non-null      float64
dtypes: float64(2), object(3)
memory usage: 168.0+ bytes


In [228]:
#conver to datetime
exp['date_added'] = pd.to_datetime(exp['date_added'], format='%Y-%m-%d')
exp['first_due'] = pd.to_datetime(exp['first_due'], format='%Y-%m-%d')

exp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       1 non-null      object        
 1   cadence     1 non-null      float64       
 2   date_added  1 non-null      datetime64[ns]
 3   first_due   1 non-null      datetime64[ns]
 4   amount      1 non-null      float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 168.0+ bytes


Nice!

Let's now calculate the next payment date. But first i'll add a count of the tracked months for reporting purposes.
https://stackoverflow.com/questions/30328427/add-months-to-a-datetime-column-in-pandas

In [229]:
#create a field that calculates the month in which you should've been saving since the last disbursement
exp['tracked_months'] = (((datetime.datetime.today().year-exp.date_added.dt.year)*12) + ((datetime.datetime.today().month-exp.date_added.dt.month))).astype(int)

In [230]:
exp

,title,cadence,date_added,first_due,amount,tracked_months
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0


### Create a Last Disbursed Date Calculation

I'll do this with a while loop through each row

In [231]:
last_paid = []
for index, row in exp.iterrows():
    payment_date = row['first_due']
    if payment_date >= datetime.datetime.today():
        last_paid.append(np.nan) 
    else:
        #while ((datetime.datetime.today()-payment_date)/np.timedelta64(1, 'M')) > row['cadence']:
        while (((datetime.datetime.today().year-payment_date.dt.year)*12) + ((datetime.datetime.today().month-payment_date.dt.month))) > row['cadence']:
            payment_date += relativedelta(months=row['cadence'])
        last_paid.append(payment_date)

In [232]:
exp['last_disbursed']  = last_paid

In [233]:
#update dtype to datetime for future formula usage
exp['last_disbursed'] = pd.to_datetime(exp['last_disbursed'], format='%m,%d,%Y')

In [234]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT


### Create a Next Due Date Calculation

I'll do this with a while loop through each row

In [235]:
due_dates = []
for index, row in exp.iterrows():
    next_date = row['first_due']
    while next_date < datetime.datetime.today():
        next_date += relativedelta(months=row['cadence'])
    due_dates.append(next_date)

In [236]:
exp['due_next'] = due_dates

In [237]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30


Now I'll calculate the months between next payment

In [238]:
#exp['months_till_due'] = ((exp.due_next - datetime.datetime.today())/np.timedelta64(1, 'M')+1).astype(int)
exp['months_till_due'] = (((exp.due_next.dt.year-exp.date_added.dt.year)*12) + ((exp.due_next.dt.month-exp.date_added.dt.month))).astype(int)

In [239]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3


Now calculate the monthly savings. There will be a differnce though for the items in which we save for the whole cadence vs. those we add with less time than the cadence. so I'll flag it if we have a full 12 months (as an example) to save vs. only 3 months left of the 12 in which we started saving. I think we need a "months to save" field

In [240]:
#updating this to not if false use months till due, but to the lesser of the cadence or the recalculated months from starting tracking to having to pay. Adding +1 to include the starting month.
exp['months_to_save'] = np.where(exp['tracked_months'] >= exp['cadence'], exp['cadence'], (((exp.due_next.dt.year-exp.date_added.dt.year)*12) + ((exp.due_next.dt.month-exp.date_added.dt.month))+1).astype(int))

In [241]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3,4.0


In [242]:
exp['monthly_sinking'] = round(exp['amount'] / exp['months_to_save'],2)
exp.replace([np.inf, -np.inf], np.nan, inplace=True)

In [243]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3,4.0,425.0


Now I need to come up with a formula that calculates how much should be saved after your deposit this month based on how many months you should've saved since the last disbursement (or since you started tracking if no disbursement yet.)


In [244]:
#exp['current_buildup'] = np.where(exp['last_disbursed'] == 0, exp['tracked_months'], ((datetime.datetime.today()-exp.last_disbursed)/np.timedelta64(1, 'M')))
exp['current_buildup'] = np.where(exp['last_disbursed'].isnull(), exp['tracked_months'], (((datetime.datetime.today().year-exp.last_disbursed.dt.year)*12) + ((datetime.datetime.today().month-exp.last_disbursed.dt.month))))

In [245]:
#convert to an integer format so it will round down to get the exact # of months.
#using fillna here to make it possible to convert if there are any NaNs
exp['current_buildup'] = exp['current_buildup'].fillna(0).astype(int)


In [246]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3,4.0,425.0,0


Now we know how many months of savings we should have done since our last disbursement in each category so let's calculate what the total balance should be as of the last transfer to savings. I'll then calculate a final balance after transfer.

In [247]:
exp['exp_current_balance'] = exp['current_buildup'] * exp['monthly_sinking']

In [248]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3,4.0,425.0,0,0.0


Now that that's there, I want to create a column that calculates the disbursement (if there is one in the current month) as a negative value. I'll then combine that into a 'final_target' with expected_savings + monthly_needed - disbursement = final value.

In [249]:
exp['current_period_disburse'] = np.where((exp['due_next'].dt.month == datetime.datetime.today().month) & (exp['due_next'].dt.year == datetime.datetime.today().year), -exp['amount'], 0)

In [250]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3,4.0,425.0,0,0.0,0.0


In [251]:
#sweet! Now I'll calculate the target ending balance
exp['target_ending_balance'] = exp['exp_current_balance'] + exp['monthly_sinking'] + exp['current_period_disburse']

In [252]:
exp

,title,cadence,date_added,first_due,amount,tracked_months,last_disbursed,due_next,months_till_due,months_to_save,monthly_sinking,current_buildup,exp_current_balance,current_period_disburse,target_ending_balance
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,0,NaT,2023-09-30,3,4.0,425.0,0,0.0,0.0,425.0


In [253]:
#one more column for indiviual item activity
exp['current_month_activity'] = exp['target_ending_balance'] - exp['exp_current_balance']

In [255]:
exp[['title', 'cadence','date_added','first_due','amount','monthly_sinking']]

,title,cadence,date_added,first_due,amount,monthly_sinking
0,After Shave,4.0,2023-06-03,2023-09-30,1700.0,425.0


### Aggregation Steps

Now that we have that all built out, we need some formulas that calculate the aggregate totals, and give some output to users after taking some inputs.

In [134]:
#I want a copy of this with a new cooler name
sinking_funds = exp.copy()

### Add ability to say which bills are due this month.

In [135]:
due_this_month = sinking_funds[sinking_funds['current_period_disburse'] < 0][['item','due_next','amount']]

In [136]:
due_dictionary = due_this_month.set_index('item').T.to_dict('list')

In [137]:
due_dictionary

{'Example (delete)': [Timestamp('2023-06-30 00:00:00'), 60]}

In [138]:
for key, value in due_dictionary.items():
    print(key, 'is due on', value[0].strftime('%m-%d-%Y'), ': $', str("{:.2f}".format(value[1])))

Example (delete) is due on 06-30-2023 : $ 60.00


### Setup way for user input of their actual current balance.

This comes in handy when there is interest building up. That is a little bit less you have to save. Or for times in which you fall behind by a couple of months.

In [139]:
#define a function to check for a valid float input
def valid_float(message):
    while True:
        try:
            count = float(input(message))
            if count >= 0:
                return count
                break
        except:
            print('You must input a numerical value.\n' + message)

In [140]:
actual_beg_balance = valid_float("Enter Current Sinking Funds Balance: ")

### Create Calculated Features and Setup Test Prints

In [141]:
expected_beginning_balance = sinking_funds['exp_current_balance'].sum()
current_period_savings = sinking_funds['monthly_sinking'].sum()
current_period_disbursements = sinking_funds['current_period_disburse'].sum()
net_period_activity = sinking_funds['current_month_activity'].sum()
target_ending_balance = sinking_funds['target_ending_balance'].sum()
actual_period_activity = target_ending_balance - actual_beg_balance
balance_adj = expected_beginning_balance - actual_beg_balance

In [142]:
print("You're expected current balance is: $" + str("{:.2f}".format(expected_beginning_balance)) + ".")
print("You're actual current balance is: $" + str("{:.2f}".format(actual_beg_balance)) + ".\n")
print('---'*10)
if actual_period_activity < 0:
    print("You should move $" + str("{:.2f}".format(-actual_period_activity)) + " from your sinking funds balance to your bill-paying account.")
else:
    print("You should move $" + str("{:.2f}".format(actual_period_activity)) + " from your checking account to your sinking funds balance.")
print("This is made up of: \n- $" + str("{:.2f}".format(current_period_savings)) + " savings for future expenses\n- $" + str("{:.2f}".format(-current_period_disbursements)) + " of disbursements for current month expense\n- $"+ str("{:.2f}".format(balance_adj)) + " of balance adjustments.\n")
print('---'*10)

print("With these changes, your target ending balance will be: $" + str("{:.2f}".format(target_ending_balance)) + ".")


You're expected current balance is: $30.00.
You're actual current balance is: $0.00.

------------------------------
You should move $0.00 from your checking account to your sinking funds balance.
This is made up of: 
- $30.00 savings for future expenses
- $60.00 of disbursements for current month expense
- $30.00 of balance adjustments.

------------------------------
With these changes, your target ending balance will be: $0.00.


In [143]:
print("-"*5 + "USER SUMMARY" + "-"*5)

-----USER SUMMARY-----


# V2 Testing
I want to remove excel from the equation and actually have this read/write from/to a json file. To do this we'll need to write functions for adding and removing records using user inputs.

Might pivot to a sqllite database for this. If I do go json, I either need to update the key and then have a subset with teh data. or have the title in the full layer with all the data and be able to add another line to this.


In [144]:
#establish possible inputs and alert
input_options = ['A','B','C','D','E']
alert = "Option not available."

In [145]:
def valid_select(message, n_choices):
    """Function for testing valid user input."""
    while True:
        try:
            answer = input(message).upper()
            if answer in input_options[:n_choices]:
                return answer
                break
            else:
                print(alert + ' Try again: ')
        except:
            print(alert + ' Try again: ')

In [146]:
#valid_select("What would you like to do?\nA) View Current Month Report\nB) Add New Expense\nC) Delete an Expense", 4)

In [147]:
def user_input_text(message):
    "Function for accepting user input of text and savings to a variable"
    var = str(input(message))
    return var

In [148]:
def valid_date(date_title):
    "Function for checking for valid date input"
    while True:
        try:
            year, month, day = map(int, input('Enter the ' + date_title + ' in YYYY-MM-DD format: ').split('-'))
            input_date = datetime.date(year, month, day)
            return input_date.strftime('%Y-%m-%d')
            break
        except:
            print('Please enter in YYYY-MM-DD format: ')
    

In [149]:
#valid_date('Due Date')

### Test specifc reading

In [150]:
#test reading
with open('../data/exp.json', 'r') as f:
    data = json.load(f)

[item for item in data['expenses'] if item['title'] == 'Costco']

[]

Might be able to use this code to get specific items
[d for d in a if d['name'] == 'pluto']


https://stackoverflow.com/questions/4391697/find-the-index-of-a-dict-within-a-list-by-matching-the-dicts-value

https://stackoverflow.com/questions/4391697/find-the-index-of-a-dict-within-a-list-by-matching-the-dicts-value

### Test Writing New Object

In [151]:
#sample varible with new entry
new_entry = {'title': 'Xmas Gifts',
 'cadence': '12',
 'date_added': '5/27/2023',
 'first_due': '12/31/2023',
 'amount': '600'}

In [152]:
#test writing to json
with open('../data/exp.json') as f:
    data = json.load(f)
    temp = data["expenses"]
    temp.append(new_entry)
    with open('../data/exp.json', 'w') as f:
        json.dump(data, f)

In [153]:
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

### Test updating specific value
As a test we'll update our Xmas Gift fund to $500 instead of $600.

In [154]:
#finding index #
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']
    index = next((index for (index, d) in enumerate(temp) if d['title'] == 'Xmas Gifts'), None)

index

3

In [155]:
#test saving down index # and writing to
with open('../data/exp.json') as f:
    data = json.load(f)
    temp = data["expenses"]
    index = next((index for (index, d) in enumerate(temp) if d['title'] == 'Xmas Gifts'), None)
    temp[index]['amount'] = 500
    with open('../data/exp.json', 'w') as f:
        json.dump(data, f)

In [156]:
#view if that worked
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

### Test Deleting Item

In [157]:
#test with read
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']
    temp[:] = [d for d in temp if d.get('title') != 'Costco']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

In [158]:
#test actual delete

#test saving down index # and writing to
with open('../data/exp.json') as f:
    data = json.load(f)
    temp = data["expenses"]
    temp[:] = [d for d in temp if d.get('title') != 'Costco']
    with open('../data/exp.json', 'w') as f:
        json.dump(data, f)

In [159]:
#view if that worked
#read updates
with open('../data/exp.json', 'r') as f:
    data = json.load(f)
    temp = data['expenses']

temp

[{'title': 'XMas Gifts',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 600.0},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Sunny',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-12-31',
  'amount': 10000.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': 500},
 {'title': 'Costco Membership',
  'cadence': 12.0,
  'date_added': '2023-06-03',
  'first_due': '2023-06-30',
  'amount': 60.0},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'},
 {'title': 'Xmas Gifts',
  'cadence': '12',
  'date_added': '5/27/2023',
  'first_due': '12/31/2023',
  'amount': '600'}]

## Define JSON Functions

In [160]:
#sample varible with new entry
new_entry = {'title': 'Xmas Gifts',
 'cadence': '12',
 'date_added': '5/27/2023',
 'first_due': '12/31/2023',
 'amount': '600'}

In [161]:
title = "Xmas Gifts"
cadence = 12
date_add = datetime.date.today().strftime('%Y-%m-%d')
first_due = datetime.date(2023,12,31).strftime('%Y-%m-%d')
amount = 600

In [162]:
exp_dict = {"title": title, "cadence": cadence, "date_added": date_add, "first_due": first_due, "amount": amount}
exp_dict

{'title': 'Xmas Gifts',
 'cadence': 12,
 'date_added': '2023-06-03',
 'first_due': '2023-12-31',
 'amount': 600}

#### Adding an expense item

In [169]:
def get_items(json_path, json_var):
    "Take in json variables and return the list of titles to reference in other functions."
    #create list of items saved to data
    with open(json_path) as f:
        data = json.load(f)
        temp = data[json_var]
        item_list = [list(dict.values())[0] for dict in temp]
    return item_list

In [183]:
def exp_in_list(message, list, match_wanted):
    "Take a user input after a prompt and return TRUE/FALSE if it is in a given list or not"
    while True:
        var = str(input(message))
        if match_wanted == False:
            if var in list:
                print('This expense already exists, please choose another title: ')
            else:
                return var
                break
        else:
            if var in list:
                return var
                break
            else:
                print('Expense not in list, please enter a valid expense: ')

In [184]:
def add_exp(json_path, json_var):
    "Take user inputs, create a dictionary, and write that dictionary to a json file."

    #setup variables for input
    title = exp_in_list("Expense Title: ", get_items(json_path, json_var), False) #validate the input isn't already in the list
    cadence = valid_float("Monthly cadence (every x months): ")
    start_date = datetime.date.today().strftime('%Y-%m-%d')
    first_due = valid_date("Due Date")
    amount = valid_float("How much do you want to save?: ")
    
    #create dictionary from variables
    exp_dict = {"title": title, "cadence": cadence, "date_added": start_date, "first_due": first_due, "amount": amount}

    with open(json_path) as f:
        data = json.load(f)
        temp = data[json_var]
        temp.append(exp_dict)
        with open(json_path, 'w') as f:
            json.dump(data, f)

In [175]:
#add_exp('../data/exp.json', 'expenses')

This expense already exists, please choose another title: 


#### Updating an Item

In [166]:
#valid_item('../data/exp.json', 'expenses', "Which expense would you like to update?: \n")

In [185]:
def update_exp(json_path, json_var):
    "Take user inputs and update the value of their choice."
    #user input validated against list of itmes
    item_to_update = exp_in_list("Which expense would you like to update?: \n",get_items(json_path, json_var), True)

    #user input on which field to update
    field_choice = valid_select("Which field would you like to update?:\nA) Title\nB) Cadence\nC) Date Added\nD) First Due\nE) Amount", 5)
    if field_choice == 'A':
        field = 'title'
    elif field_choice == 'B':
        field = 'cadence'
    elif field_choice == 'C':
        field = 'date_added'
    elif field_choice == 'D':
        field = 'first_due'
    elif field_choice == 'E':
        field = 'amount'

    #user input to update field, using an if statement to decice which input validation to use
    if field == ('title'):
        new_value = user_input_text("New Title: ")
    elif field in ('cadence', 'amount'):
        new_value = valid_float("New Value: ")
    else:
        new_value = valid_date("New Date: ")
    
    #write to JSON
    with open(json_path) as f:
        data = json.load(f)
        temp = data[json_var]
        index = next((index for (index, d) in enumerate(temp) if d['title'] == item_to_update), None)
        temp[index][field] = new_value
        with open('../data/exp.json', 'w') as f:
            json.dump(data, f)

In [182]:
#update_exp('../data/exp.json', 'expenses')

Expense not in list, please enter a valid expense: 


### Deleting and Item

In [194]:
def delete_expense(json_path, json_var):
    item_to_delete = exp_in_list("Which expense would you like to delete?: \n",get_items(json_path, json_var), True)
    
    with open(json_path, 'r') as f:
        data = json.load(f)
        temp = data[json_var]
        temp[:] = [d for d in temp if d.get('title') != item_to_delete]
        with open(json_path, 'w') as f:
            json.dump(data, f)

In [188]:
#delete_expense('../data/exp.json', 'expenses')

Expense not in list, please enter a valid expense: 


## Test Full Sequence of Functions

In [190]:
#add_exp('../data/exp.json', 'expenses')

In [191]:
#add_exp('../data/exp.json', 'expenses')

In [192]:
#update_exp('../data/exp.json', 'expenses')

In [193]:
#delete_expense('../data/exp.json', 'expenses')

## Read from JSON into DF

In [265]:
test_df = pd.json_normalize(pd.read_json('../data/exp.json')['expenses'])

### Print Active Expenses

In [266]:
def view_expenses(df, index_col):
    print("Active Expenses: \n")
    if len(df) == 0:
        print('There are no active expenses.')
    else:
        print(df.set_index(index_col))

In [267]:
len(test_df)

0

In [272]:
if test_df.empty == True:
    print(1)
else:
    print(0)

1


In [268]:
view_expenses(test_df, 'title')

Active Expenses: 

There are no active expenses.


## Main Menu Options Test

In [200]:
#just a test of the logic, will change field variable setting to running functions in production fiel

menu_choice = valid_select("What function would you like to perform?:\nA) Run Monthly Report\nB) View Active Expenses\nC) Add New Expense\nD) Update Expense Item\nE) Delete Expense", 5)
if menu_choice == 'A':
    test = 'title'
elif menu_choice == 'B':
    test = 'cadence'
elif menu_choice == 'C':
    test = 'date_added'
elif menu_choice == 'D':
    test = 'first_due'
elif menu_choice == 'E':
    test = 'amount'

In [201]:
test

'title'